In [2]:
##Attempt at using sigkernel with data
import apache_beam

In [3]:
import os
import weatherbench2
import xarray as xr
import numpy as np
import math
from weatherbench2.regions import SliceRegion, ExtraTropicalRegion
from weatherbench2.evaluation import evaluate_in_memory
from weatherbench2 import config

In [4]:
forecast_path = 'gs://weatherbench2/datasets/ifs_ens/2018-2022-64x32_equiangular_conservative.zarr'
obs_path = 'gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr'

In [5]:
forecasts = xr.open_zarr(forecast_path)
observations = xr.open_zarr(obs_path)

In [ ]:
#Goal is to have batch as model, length x as time, value as lat/long flattened

In [41]:
#Test with pred_timedelta = 0

forecastssel = forecasts['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-01-10'))
forecasts0 = forecastssel[:,:,0,:,:]
savedfor = forecasts0.values

In [42]:
observationsel = observations['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-01-10'))
savedobs = observationsel[::2,:,:].values

In [43]:
savedobs.shape
reshabedobs = savedobs.reshape(-1,64*32)
reshabedobs.shape

#print(savedobs[0,1,0])  #2d position flattened to 1d, go through every latitude for 0 long, then every lat for 1 long etc.
#print(reshabedobs[0,32])
csrsobs = np.cumsum(reshabedobs,axis=0) #Culmative sum goes elementwise 

In [44]:
savedfor.shape
reshavedfors = savedfor.reshape(20,50,64*32)
csrsfor = np.cumsum(reshavedfors,axis=0)

#print(savedfor[0,2,0,1]+savedfor[1,2,0,1]+savedfor[2,2,0,1])
#csrsfor[2,2,1]
csrsfor.shape


(20, 50, 2048)

In [45]:
print("Obs shape:", csrsobs.shape)
print("Fors shape:", csrsfor.shape)

csrsobsex = np.expand_dims(csrsobs, axis = 0)
csrsforsw = np.transpose(csrsfor, (1,0,2))


print("Final Obs shape:", csrsobsex.shape)
print("Final Fors shape:", csrsforsw.shape)

print(csrsobsex.mean())
print(csrsforsw.mean())


Obs shape: (20, 2048)
Fors shape: (20, 50, 2048)
Final Obs shape: (1, 20, 2048)
Final Fors shape: (50, 20, 2048)
566411.25
566381.0


In [11]:
csrsobs

array([[  49846.934,   50196.53 ,   50391.094, ...,   49238.684,
          48659.176,   48309.336],
       [  99582.08 ,  100317.56 ,  100602.34 , ...,   98387.58 ,
          97347.555,   96948.   ],
       [ 149389.56 ,  150369.94 ,  150675.31 , ...,  147365.78 ,
         146372.73 ,  145847.44 ],
       ...,
       [ 902296.25 ,  908305.7  ,  910008.2  , ...,  887298.8  ,
         889337.25 ,  888369.3  ],
       [ 953454.6  ,  959792.7  ,  961143.75 , ...,  937034.1  ,
         939557.7  ,  939232.4  ],
       [1004562.3  , 1011216.9  , 1012094.06 , ...,  986696.44 ,
         989878.25 ,  989886.   ]], dtype=float32)

In [12]:
import numpy as np
import sigkernel
import torch
from einops import rearrange
from itertools import product
import cython

In [13]:
## Date processed completed, beggining
X = torch.tensor(csrsforsw, dtype=torch.double)
y = torch.tensor(csrsobsex, dtype=torch.double)

In [21]:
static_kernel = sigkernel.RBFKernel(sigma=10000)
dyadic_order = 2
signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

K_XX = signature_kernel.compute_Gram(X,X)
K_Xy = signature_kernel.compute_Gram(X, y)
K_XX_m = (torch.sum(K_XX) - torch.sum(torch.diag(K_XX))) / (K_XX.shape[0] * (K_XX.shape[0] - 1.))
print(K_XX)
print(K_Xy) #
print("K_XX_m:", K_XX_m) #Expected self similirity of predictions... 
print(-2. * torch.mean(K_Xy)) #Expected similarity between prediction and observation
print(K_XX_m -(2.*torch.mean(K_Xy)))

#Feels like there are rounding errors

tensor([[1.5280e+05, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 1.5278e+05, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.5278e+05,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        ...,
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.5276e+05, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 1.5280e+05,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00,
         1.5300e+05]], dtype=torch.float64)
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
     

In [ ]:
static_kernel = sigkernel.Linear_ID_Kernel() #LinearKernel or Linear_ID_Kernel goes to infinity
dyadic_order = 2
signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

K_XX = signature_kernel.compute_Gram(X,X)
K_Xy = signature_kernel.compute_Gram(X, y)
K_XX_m = (torch.sum(K_XX) - torch.sum(torch.diag(K_XX))) / (K_XX.shape[0] * (K_XX.shape[0] - 1.))
print(K_XX)
print(K_Xy) #
print("K_XX_m:", K_XX_m) #Expected self similirity of predictions... 
print(-2. * torch.mean(K_Xy)) #Expected similarity between prediction and observation
print(K_XX_m -(2.*torch.mean(K_Xy)))


tensor([[inf, inf, inf,  ..., inf, inf, inf],
        [inf, inf, inf,  ..., inf, inf, inf],
        [inf, inf, inf,  ..., inf, inf, inf],
        ...,
        [inf, inf, inf,  ..., inf, inf, inf],
        [inf, inf, inf,  ..., inf, inf, inf],
        [inf, inf, inf,  ..., inf, inf, inf]], dtype=torch.float64)
tensor([[inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],
        [inf],

In [22]:
signature_kernel.compute_scoring_rule(X,y)

tensor(-1., dtype=torch.float64)

In [ ]:
#Trying a mean shift? #Used minimum, didn't work

forecastssel = forecasts['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-01-10'))
forecasts0 = forecastssel[:,:,0,:,:]
savedfor = forecasts0.values

observationsel = observations['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-01-10'))
savedobs = observationsel[::2,:,:].values

#Shifting everything down by minimum value
minvalue = np.array([savedobs.min(),savedfor.min()]).min()
print(minvalue)
print(savedobs.mean())
savedobsshift = savedobs - minvalue
savedforshift = savedfor - minvalue

savedobsshift.shape
reshabedobs = savedobsshift.reshape(-1,64*32)
reshabedobs.shape

# #print(savedobs[0,1,0])  #2d position flattened to 1d, go through every latitude for 0 long, then every lat for 1 long etc.
# #print(reshabedobs[0,32])
csrsobs = np.cumsum(reshabedobs,axis=0) #Culmative sum goes elementwise 

# savedfor.shape
reshavedfors = savedforshift.reshape(20,50,64*32)
csrsfor = np.cumsum(reshavedfors,axis=0)

# #print(savedfor[0,2,0,1]+savedfor[1,2,0,1]+savedfor[2,2,0,1])
# #csrsfor[2,2,1]
# csrsfor.shape

print("Obs shape:", csrsobs.shape)
print("Fors shape:", csrsfor.shape)

csrsobsex = np.expand_dims(csrsobs, axis = 0)
csrsforsw = np.transpose(csrsfor, (1,0,2))

print("Final Obs shape:", csrsobsex.shape)
print("Final Fors shape:", csrsforsw.shape)



45149.05
53974.15
Obs shape: (20, 2048)
Fors shape: (20, 50, 2048)
Final Obs shape: (1, 20, 2048)
Final Fors shape: (50, 20, 2048)
92346.23
92315.99
(1, 20, 2048)


In [53]:
X = torch.tensor(csrsforsw, dtype=torch.double)
y = torch.tensor(csrsobsex, dtype=torch.double)

static_kernel = sigkernel.RBFKernel(sigma=2)
dyadic_order = 2
signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

K_XX = signature_kernel.compute_Gram(X,X)
K_Xy = signature_kernel.compute_Gram(X, y)
K_XX_m = (torch.sum(K_XX) - torch.sum(torch.diag(K_XX))) / (K_XX.shape[0] * (K_XX.shape[0] - 1.))
print(K_XX)
print(K_Xy) #
print("K_XX_m:", K_XX_m) #Expected self similirity of predictions... 
print(-2. * torch.mean(K_Xy)) #Expected similarity between prediction and observation
print(K_XX_m -(2.*torch.mean(K_Xy)))

tensor([[8.7792e+04, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 8.3319e+04, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 8.4337e+04,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        ...,
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.1022e+04, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 8.2747e+04,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00,
         1.0720e+05]], dtype=torch.float64)
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
     

In [ ]:
#Trying a divid by maximum shift? Does generate real value: does it work? 

forecastssel = forecasts['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-01-10'))
forecasts0 = forecastssel[:,:,0,:,:]
savedfor = forecasts0.values

observationsel = observations['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-01-10'))
savedobs = observationsel[::2,:,:].values

#Shifting everything down by minimum value
maxvalue = np.array([savedobs.max(),savedfor.max()]).max()
print(maxvalue)
print(savedobs.mean())
savedobsshift = savedobs/maxvalue
savedforshift = savedfor/maxvalue

savedobsshift.shape
reshabedobs = savedobsshift.reshape(-1,64*32)
reshabedobs.shape

# #print(savedobs[0,1,0])  #2d position flattened to 1d, go through every latitude for 0 long, then every lat for 1 long etc.
# #print(reshabedobs[0,32])
csrsobs = np.cumsum(reshabedobs,axis=0) #Culmative sum goes elementwise 

# savedfor.shape
reshavedfors = savedforshift.reshape(20,50,64*32)
csrsfor = np.cumsum(reshavedfors,axis=0)

# #print(savedfor[0,2,0,1]+savedfor[1,2,0,1]+savedfor[2,2,0,1])
# #csrsfor[2,2,1]
# csrsfor.shape

print("Obs shape:", csrsobs.shape)
print("Fors shape:", csrsfor.shape)

csrsobsex = np.expand_dims(csrsobs, axis = 0)
csrsforsw = np.transpose(csrsfor, (1,0,2))

print("Final Obs shape:", csrsobsex.shape)
print("Final Fors shape:", csrsforsw.shape)

58347.312
53974.15
Obs shape: (20, 2048)
Fors shape: (20, 50, 2048)
Final Obs shape: (1, 20, 2048)
Final Fors shape: (50, 20, 2048)


In [57]:
X = torch.tensor(csrsforsw, dtype=torch.double)
y = torch.tensor(csrsobsex, dtype=torch.double)

static_kernel = sigkernel.RBFKernel(sigma=2)
dyadic_order = 2
signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

K_XX = signature_kernel.compute_Gram(X,X)
K_Xy = signature_kernel.compute_Gram(X, y)
K_XX_m = (torch.sum(K_XX) - torch.sum(torch.diag(K_XX))) / (K_XX.shape[0] * (K_XX.shape[0] - 1.))
print(K_XX)
print(K_Xy) #
print("K_XX_m:", K_XX_m) #Expected self similirity of predictions... 
print(-2. * torch.mean(K_Xy)) #Expected similarity between prediction and observation
print(K_XX_m -(2.*torch.mean(K_Xy)))

tensor([[153677.1668, 141831.0625, 145261.9578,  ..., 142337.7196,
         145606.2026, 146748.5873],
        [141831.0625, 153677.1669, 145893.8163,  ..., 147020.9039,
         142580.1742, 144588.5203],
        [145261.9578, 145893.8163, 153677.1669,  ..., 145056.4159,
         145686.2982, 146244.4211],
        ...,
        [142337.7196, 147020.9039, 145056.4159,  ..., 153677.1669,
         142756.5391, 144857.7361],
        [145606.2026, 142580.1742, 145686.2982,  ..., 142756.5391,
         153677.1668, 145791.2138],
        [146748.5873, 144588.5203, 146244.4211,  ..., 144857.7361,
         145791.2138, 153677.1675]], dtype=torch.float64)
tensor([[111963.1239],
        [113400.2121],
        [112101.8209],
        [111965.0228],
        [112209.0951],
        [111869.1087],
        [111856.3950],
        [115466.1790],
        [113719.4448],
        [112118.7457],
        [113137.7077],
        [111740.7530],
        [111712.5862],
        [111781.7241],
        [111070.7840],
  